# Week 5 Workshop: Univariate Analysis
## EDA Part 1: Analyzing Water Consumption Data

**Student Name:** (Your name here)  
**Date:** (Today's date)  
**Dataset:** Water Consumption (HISTORICO_CONSUMO) from datos.gov.co

---

### Instructions

Apply the **5-step univariate analysis framework** to each of the 3 variables below:

1. **CONSUMO_FACTURADO** - Billed consumption (m3)
2. **VALOR_FACTURADO** - Billed amount (COP)
3. **NUMERO_SUSCRIPTORES** - Number of subscribers

For each variable, complete all 5 steps:
- Identify (data type, missing values)
- Summarize (mean, median, mode)
- Spread (std, IQR)
- Visualize (histogram, boxplot)
- Detect (outliers using IQR method)

---

## Setup

In [ ]:
# Import libraries
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns
from scipy import stats

# Display settings
pd.set_option('display.max_columns', None)
pd.set_option('display.float_format', '{:,.2f}'.format)

# Plot style
plt.style.use('seaborn-v0_8-whitegrid')
plt.rcParams['figure.figsize'] = (12, 5)

In [ ]:
# Load the dataset
url = "https://www.datos.gov.co/api/views/wcpc-hgdr/rows.csv?accessType=DOWNLOAD"
df = pd.read_csv(url)

# Quick overview
print(f"Dataset shape: {df.shape}")
print(f"\nColumns:\n{df.columns.tolist()}")
df.head()

In [ ]:
# Check data types and missing values
df.info()

---

## Helper Function (Use this for all variables)

Run this cell to define a reusable analysis function.

In [ ]:
def univariate_analysis(df, var):
    """
    Perform complete univariate analysis on a variable.
    
    Parameters:
    -----------
    df : pandas DataFrame
        The dataset
    var : str
        The column name to analyze
        
    Returns:
    --------
    dict : Summary statistics
    """
    print("=" * 60)
    print(f"UNIVARIATE ANALYSIS: {var}")
    print("=" * 60)
    
    # Step 1: Identify
    print(f"\n--- STEP 1: IDENTIFY ---")
    print(f"Data type: {df[var].dtype}")
    print(f"Total records: {len(df):,}")
    print(f"Non-null values: {df[var].count():,}")
    print(f"Missing values: {df[var].isna().sum():,} ({df[var].isna().sum()/len(df)*100:.2f}%)")
    
    # Step 2: Summarize (Central Tendency)
    print(f"\n--- STEP 2: CENTRAL TENDENCY ---")
    mean_val = df[var].mean()
    median_val = df[var].median()
    mode_val = df[var].mode()[0] if len(df[var].mode()) > 0 else np.nan
    ratio = mean_val / median_val if median_val != 0 else np.nan
    
    print(f"Mean: {mean_val:,.2f}")
    print(f"Median: {median_val:,.2f}")
    print(f"Mode: {mode_val:,.2f}")
    print(f"Mean/Median ratio: {ratio:.2f}")
    
    # Step 3: Spread (Dispersion)
    print(f"\n--- STEP 3: DISPERSION ---")
    std_val = df[var].std()
    var_val = df[var].var()
    q1 = df[var].quantile(0.25)
    q3 = df[var].quantile(0.75)
    iqr = q3 - q1
    
    print(f"Standard Deviation: {std_val:,.2f}")
    print(f"Variance: {var_val:,.2f}")
    print(f"Q1 (25th): {q1:,.2f}")
    print(f"Q3 (75th): {q3:,.2f}")
    print(f"IQR: {iqr:,.2f}")
    print(f"Range: {df[var].min():,.2f} to {df[var].max():,.2f}")
    
    # Step 4: Visualize
    print(f"\n--- STEP 4: VISUALIZATION ---")
    fig, axes = plt.subplots(1, 2, figsize=(14, 5))
    
    # Histogram
    axes[0].hist(df[var].dropna(), bins=50, edgecolor='black', alpha=0.7, color='steelblue')
    axes[0].axvline(mean_val, color='red', linestyle='--', linewidth=2, label=f'Mean: {mean_val:,.0f}')
    axes[0].axvline(median_val, color='green', linestyle='-', linewidth=2, label=f'Median: {median_val:,.0f}')
    axes[0].set_xlabel(var, fontsize=12)
    axes[0].set_ylabel('Frequency', fontsize=12)
    axes[0].set_title(f'Distribution of {var}', fontsize=14)
    axes[0].legend()
    
    # Boxplot
    axes[1].boxplot(df[var].dropna(), vert=True, patch_artist=True,
                    boxprops=dict(facecolor='steelblue', alpha=0.7))
    axes[1].set_ylabel(var, fontsize=12)
    axes[1].set_title(f'Box Plot of {var}', fontsize=14)
    
    plt.tight_layout()
    plt.show()
    
    # Step 5: Detect Outliers
    print(f"\n--- STEP 5: OUTLIER DETECTION (IQR Method) ---")
    lower_bound = q1 - 1.5 * iqr
    upper_bound = q3 + 1.5 * iqr
    outliers = df[(df[var] < lower_bound) | (df[var] > upper_bound)]
    lower_outliers = df[df[var] < lower_bound]
    upper_outliers = df[df[var] > upper_bound]
    
    print(f"Lower bound: {lower_bound:,.2f}")
    print(f"Upper bound: {upper_bound:,.2f}")
    print(f"Total outliers: {len(outliers):,} ({len(outliers)/len(df)*100:.2f}%)")
    print(f"  - Lower outliers: {len(lower_outliers):,}")
    print(f"  - Upper outliers: {len(upper_outliers):,}")
    
    # Distribution type
    if ratio > 1.2:
        dist_type = "Right-skewed"
        best_measure = "Median"
    elif ratio < 0.8:
        dist_type = "Left-skewed"
        best_measure = "Median"
    else:
        dist_type = "Approximately symmetric"
        best_measure = "Mean"
    
    print(f"\n--- SUMMARY ---")
    print(f"Distribution type: {dist_type}")
    print(f"Recommended central measure: {best_measure}")
    print("=" * 60)
    
    return {
        'variable': var,
        'mean': mean_val,
        'median': median_val,
        'mode': mode_val,
        'std': std_val,
        'iqr': iqr,
        'distribution': dist_type,
        'outliers_pct': len(outliers)/len(df)*100,
        'best_measure': best_measure
    }

---

# Variable 1: CONSUMO_FACTURADO

**Description:** Billed water consumption in cubic meters (m3)

In [ ]:
# TODO: Run the univariate analysis for CONSUMO_FACTURADO
results_consumo = univariate_analysis(df, 'CONSUMO_FACTURADO')

### Your Interpretation for CONSUMO_FACTURADO

**Distribution type:** (Write your answer - Normal, Right-skewed, Left-skewed, or Bimodal)

**Explanation:** (Why does it have this distribution? What does it mean in context?)

**Outlier investigation:** (What might cause the outliers? Should we keep or remove them?)

**One-sentence summary:** (Summarize your key finding)

---

# Variable 2: VALOR_FACTURADO

**Description:** Billed amount in Colombian Pesos (COP)

In [ ]:
# TODO: Run the univariate analysis for VALOR_FACTURADO
results_valor = univariate_analysis(df, 'VALOR_FACTURADO')

### Your Interpretation for VALOR_FACTURADO

**Distribution type:** (Write your answer)

**Explanation:** (Why does it have this distribution?)

**Comparison to CONSUMO_FACTURADO:** (Are the patterns similar? Why or why not?)

**One-sentence summary:** (Summarize your key finding)

---

# Variable 3: NUMERO_SUSCRIPTORES

**Description:** Number of subscribers (users) in each municipality

In [ ]:
# TODO: Run the univariate analysis for NUMERO_SUSCRIPTORES
results_suscriptores = univariate_analysis(df, 'NUMERO_SUSCRIPTORES')

### Your Interpretation for NUMERO_SUSCRIPTORES

**Distribution type:** (Write your answer)

**Explanation:** (What does this distribution tell us about municipalities in the dataset?)

**Outlier investigation:** (What might the high-subscriber outliers represent?)

**One-sentence summary:** (Summarize your key finding)

---

# GroupBy: Compare Statistics Across Groups

Now that you have analyzed each variable individually, use **GroupBy** to see how the statistics change across groups.

**Concept:** GroupBy splits your data by a category, applies a statistic to each group, and combines the results. Think of sorting M&Ms by color and counting each pile.

**Pattern:** `df.groupby('GROUP_COLUMN')['VALUE_COLUMN'].operation()`

### Task 1: Mean Consumption by Use Type (USO)

Calculate the mean CONSUMO_FACTURADO for each use type. Which type consumes the most water?

In [ ]:
# TODO: Calculate mean consumption by use type
# Hint: df.groupby('USO')['CONSUMO_FACTURADO'].mean()


### Task 2: Median Consumption by Stratum (ESTRATO)

Calculate the median CONSUMO_FACTURADO for each socioeconomic stratum. Does consumption increase with stratum?

In [ ]:
# TODO: Calculate median consumption by stratum
# Hint: df.groupby('ESTRATO')['CONSUMO_FACTURADO'].median()


### Task 3: Count and Total by Department

How many records per department? Which department has the highest total consumption?

In [ ]:
# TODO: Count records per department
# Hint: df.groupby('DEPARTAMENTO')['CONSUMO_FACTURADO'].count().sort_values(ascending=False)


# TODO: Total consumption per department (top 5)
# Hint: df.groupby('DEPARTAMENTO')['CONSUMO_FACTURADO'].sum().sort_values(ascending=False).head(5)


### GroupBy Reflection

**Question:** Compare the overall mean consumption (from Variable 1 analysis) with the group means by USO. What insight does GroupBy reveal that a single statistic hides?

**Your answer:** (Write your answer here)

---

# Final Summary Table

Compile all your findings into a summary table.

In [ ]:
# TODO: Create the summary table using your results
summary_data = [
    results_consumo,
    results_valor,
    results_suscriptores
]

summary_df = pd.DataFrame(summary_data)
summary_df = summary_df[['variable', 'mean', 'median', 'distribution', 'outliers_pct', 'best_measure']]
summary_df.columns = ['Variable', 'Mean', 'Median', 'Distribution', 'Outliers (%)', 'Best Measure']

print("\n" + "=" * 80)
print("FINAL SUMMARY TABLE")
print("=" * 80)
print(summary_df.to_string(index=False))
print("=" * 80)

---

# Reflection Questions

Answer these questions based on your analysis:

### 1. Pattern Comparison
Do all three variables follow similar distribution patterns? Why might that be?

**Your answer:**

### 2. Mean vs Median
For which variable is the difference between mean and median the largest? What does this indicate?

**Your answer:**

### 3. Outlier Decision
If you had to decide whether to keep or remove outliers for further analysis, what would you choose for each variable and why?

**Your answer:**

### 4. Next Steps
Based on your univariate analysis, what bivariate questions would you like to explore in Week 7?

**Your answer:**

---

## Submission Checklist

Before submitting, verify that you have:

- [ ] Completed analysis for all 3 variables
- [ ] Added interpretations for each variable
- [ ] Created the final summary table
- [ ] Answered all reflection questions
- [ ] Run all cells to ensure no errors
- [ ] Saved the notebook

---

*Week 5 Workshop - Data Analytics Course*